In [1]:
import gensim
#model = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
#model = gensim.models.Word2Vec.load("../models/custom_w2v")
model = gensim.models.Word2Vec.load("../models/custom_w2v_intersect_GoogleNews")
model.init_sims(replace=True) #precomputed l2 normed vectors in-place – saving the extra RAM

In [2]:
model_vectors = model.wv

In [3]:
from GAIL import get_cosine_sim, get_action, get_raw_action
import torch

In [31]:
d = torch.load('../dat/processed/padded_vectorized_states_v3.pt')
#d = torch.load('../dat/processed/vectorized_states_v3.pt')
raw = torch.load('../dat/processed/raw_states_v3.pt')

In [32]:
for index, vects in d.items():
    # each is N x 300
    input_state, next_state = vects[0], vects[1]
    # raw strings corresponding to embeddings
    raw_input_state, raw_next_state = list(raw.keys())[index], raw[list(raw.keys())[index]]
    print(raw_input_state)
    print(input_state)
    print(raw_next_state)
    print(next_state)
    if index > 1:
        break

go find one .
tensor([ 51, 217, 105,   0,   0])
<person> 's name .
tensor([  5,   8, 170,   0,   0])


In [33]:
s1 = torch.rand(size = (5, 50))
s2 = torch.rand(size = (5, 50))
get_cosine_sim(s1, s2)

0.9343489

In [34]:
import torch.nn as nn
import gensim
TOKENS_RAW_CUTOFF = 5

def get_vectors():
    model = gensim.models.Word2Vec.load("../models/custom_w2v_intersect_GoogleNews") # ("./models/custom_w2v")
    model.init_sims(replace=True) #precomputed l2 normed vectors in-place – saving the extra RAM
    return torch.FloatTensor(model.wv.vectors)

def from_pretrained(embeddings=None, freeze=True):
    if not embeddings:
        embeddings = get_vectors() # 2 D embeddings param
    rows, cols = embeddings.shape
    # A simple lookup table that stores embeddings of a fixed dictionary and size.
    embedding = torch.nn.Embedding(num_embeddings=rows, embedding_dim=cols)
    embedding.weight = torch.nn.Parameter(embeddings)
    # no update
    embedding.weight.requires_grad = not freeze
    return embedding

In [ ]:
class DecRnn(nn.Module):
        def __init__(self, hidden_size, num_layers,
                 device='cpu', drop_prob=0, lstm=True, feature_norm=False,
                 input_size=300, output_size=300,
                 bidirectional=False):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device

        self.embedding = from_pretrained()

        self.memory_cell = torch.nn.GRU(input_size=(hidden_size*2)+hidden_size,
                                hidden_size=hidden_size,
                                num_layers=num_layers,
                                batch_first=True,
                                # make dropout 0 if num_layers is 1
                                dropout=drop_prob * (num_layers != 1),
                                bidirectional=bidirectional)
    
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        #mask = [batch size, src len]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs, mask)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [78]:
class EncRnn(nn.Module):

    def __init__(self, hidden_size, num_layers,
                 device='cpu', drop_prob=0, lstm=True, feature_norm=False,
                 input_size=300,
                 bidirectional=True):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device

        self.embedding = from_pretrained()

        self.memory_cell = torch.nn.GRU(input_size=input_size,
                                hidden_size=hidden_size,
                                num_layers=num_layers,
                                batch_first=True,
                                # make dropout 0 if num_layers is 1
                                dropout=drop_prob * (num_layers != 1),
                                bidirectional=bidirectional)
        self.linear = nn.Linear(hidden_size * 2, hidden_size)
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x):
        print(x.shape)
        x = self.dropout(self.embedding(x))
        # packing for computation and performance
        packed_x = torch.nn.utils.rnn.pack_padded_sequence(x, batch_first=True, lengths = [x.shape[1]]*x.shape[0])
        out, hidden = self.memory_cell(packed_x)
        out, _ = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True) # unpack
        out = out.transpose(1,0)
        print(out.shape, hidden.shape)
        # initial decoder hidden is final hidden state of the forwards and
        # backwards encoder RNNs fed through a linear layer
        concated = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        hidden = torch.tanh(self.linear(concated))
        print(out.shape, hidden.shape)
        return out, hidden

In [81]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        
        self.attn = nn.Linear((hidden_size * 2) + hidden_size, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10) # !!! 0 lines up with our padded value so this works for now
        
        return F.softmax(attention, dim = 1)

In [79]:
actor = EncRnn(hidden_size=2, num_layers=2, input_size=300)#, output_size=50) #Seq2Seq(hidden_size=2, num_layers=2, input_size=50, output_size=50)
actor_criterion = torch.nn.MSELoss()
actor_optimizer = torch.optim.Adam(actor.parameters(), lr=0.001)
#print(actor)

running_loss=0.0
for idx, (index, vects) in enumerate(d.items()):
    # each is N x 300
    input_state, next_state = vects[0], vects[1]
    # raw strings corresponding to embeddings
    raw_input_state, raw_next_state = list(raw.keys())[index], raw[list(raw.keys())[index]]
    
    mu = actor(input_state.unsqueeze(0))
    
    # ACTOR FORMAT
    logstd = torch.zeros_like(mu.detach())
    std = 0.05*torch.exp(logstd)
    action = get_action(mu.detach(), std)[0]
    
    print(raw_input_state," || ",get_raw_action(action))
    # non greedy and greedy sim
    print(raw_next_state, 'non-greedy:', get_cosine_sim(next_state, action), 'greedy:', get_cosine_sim(next_state, action, type='greedy'))
    print("\n")
    # CONVENTIONAL TRAINING
    # zero the parameter gradients
    actor_optimizer.zero_grad()
    # forward + backward + optimize
    if idx == 0:
        print(mu.shape, next_state.shape)
    loss = actor_criterion(mu.squeeze(0), next_state)
    loss.backward()
    actor_optimizer.step()

    # print statistics
    running_loss += loss.item()
    running_loss = 0.0

    if idx > 100:
        break

torch.Size([1, 5])
torch.Size([5, 1, 4]) torch.Size([4, 1, 2])
torch.Size([5, 1, 4]) torch.Size([1, 2])


AttributeError: 'tuple' object has no attribute 'detach'

In [25]:
#dir(model)

In [69]:
import spacy

In [71]:
nlp = spacy.load('./spacy-blank-GoogleNews/')

In [ ]:
for token_vector in action:
    v = token_vector.numpy()
    nlp.similarity(v, )

In [72]:
from numba import jit

@jit(nopython=True)
def cosine_similarity_numba(u:np.ndarray, v:np.ndarray):
    assert(u.shape[0] == v.shape[0])
    uv = 0
    uu = 0
    vv = 0
    for i in range(u.shape[0]):
        uv += u[i]*v[i]
        uu += u[i]*u[i]
        vv += v[i]*v[i]
    cos_theta = 1
    if uu != 0 and vv != 0:
        cos_theta = uv/np.sqrt(uu*vv)
    return cos_theta

In [73]:
def most_similar(word, topn=5):
  word = nlp.vocab[str(word)]
  queries = [
      w for w in word.vocab 
      if w.is_lower == word.is_lower and w.prob >= -15 and np.count_nonzero(w.vector)
  ]

  by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=True)
  return [(w.lower_,w.similarity(word)) for w in by_similarity[:topn+1] if w.lower_ != word.lower_]

most_similar("dog", topn=3)

[]